# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import csv
import logging
import os

import joblib
import numpy as np
import pandas as pd
import pickle
import pkg_resources

import azureml.core
from azureml.core import Dataset, Experiment, Workspace
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize workspace from config

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Azure ML Experiment

Create and name the experiment

In [ ]:
# Choose a name for experiment
experiment_name = 'automl-heart-failure'
project_folder = './heartfailure'

experiment = Experiment(ws, experiment_name)

## Create or Attach an Auto ML Compute Cluster
Search for existing compute cluster; if not found, create one

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# If using a different existing cluster, enter the name in place of 'mlecscompute' below:
amlcompute_cluster_name = 'mlecscompute'

# Verify cluster existence: 
try: 
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found; creating new cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_v2', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


Search for uploaded dataset in AzureML Studio; if not found, find it through the URL

In [ ]:
# Attempt to load the dataset from the Workspace, but otherwise, create from file
# dataset file located at 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
found = False
# key should be set to 'heartfailuredataset' 
key = 'heartfailuredataset'
# description_text should be set to 'heartfailuredataset' 
description_text = 'heartfailuredataset'

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    datasetfile = 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(datasetfile)
    dataset = dataset.register(workspace = ws, name = key, description = description_text)

df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# AutoML settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# AutoML config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# Submit experiment
automl_run = experiment.submit(config = automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, fitted_automl_model = automl_run.get_output()
print(best_automl_run)
print(fitted_automl_model)

best_automl_run.get_metrics()

joblib.dump(fitted_automl_model, 'automlheartmodel')

## Model Deployment

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service